In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import argrelextrema


from src.stroop_analysis import plot_stroop_stacceleration
from src.accelerometer import plot_fourier_transformation

In [ ]:
stroop_df = pd.read_csv('stroop_sessions.csv')

In [ ]:
stroop_df.head(5)

In [ ]:
stroop_df.describe()

In [ ]:
single_session_df = stroop_df[stroop_df['uuid'] == 'd3eac1ae-1cb0-4b13-b631-f992ee28c732']

In [ ]:
stroop_df['click_number'].unique()


In [ ]:
plot_stroop_stacceleration(single_session_df, 'test session')

In [ ]:
plot_fourier_transformation(single_session_df, 'test session')

In [ ]:
df_std = stroop_df.groupby(['uuid'])['x', 'y', 'z', 'mag'].agg('std')
df_std.boxplot()

In [ ]:
def plot_df_acceleration(data):
    data.drop(columns=['click_number','duration','time_stamp', 'Unnamed: 0']).plot(figsize=(20,30), grid=True, subplots=True, legend=True, ylim=[-2500,6000])

In [ ]:
stroop_df.sort_index(inplace=True)

In [ ]:
plot_df_acceleration(stroop_df)

In [ ]:
def count_peaks(data, column):
    ilocs_min = argrelextrema(data[column].values, np.less_equal, order=3)[0]
    ilocs_max = argrelextrema(data[column].values, np.greater_equal, order=3)[0]
    return (len(ilocs_min) + len(ilocs_max))

def add_subject_session_peaks(subject_peaks,df_session, column):
    overall_session_peaks = count_peaks(df_session, column)
    subject = df_session.iloc[0, df_session.columns.get_loc("subject")]
    if not subject in subject_peaks:
        subject_peaks[subject] = []
    peaks = count_peaks(df_session, column)
    subject_peaks[subject].append(peaks)

def add_click_session_peaks(click_peaks,clicks,df_session, column):
    overall_session_peaks = count_peaks(df_session, column)
    for click in clicks:
        df_click = df_session[df_session['click_number'] == click]
        if not click in click_peaks:
            click_peaks[click] = []
        peaks = count_peaks(df_click, column)
        click_peaks[click].append(peaks)

def peak_analysis(data, column):

    subject_peaks = {}
    overall_peaks = count_peaks(data, column)
    click_peaks = {}

    clicks = data['click_number'].unique()

    for session in data['uuid'].unique():
        session_subject_peaks = []
        session_click_peaks = []
        df_session = data[data['uuid'] == session]
        add_subject_session_peaks(subject_peaks, df_session, column)
        add_click_session_peaks(click_peaks, clicks, df_session, column)

    return {
       'overall_peaks':overall_peaks,
       'subject_peaks':subject_peaks,
       'click_peaks':click_peaks
    }

In [ ]:
peak_analysis_x = peak_analysis(stroop_df,'x')
peak_analysis_y = peak_analysis(stroop_df,'y')
peak_analysis_z = peak_analysis(stroop_df,'z')
peak_analysis_mag = peak_analysis(stroop_df,'mag')

In [ ]:
def plot_peak_analysis(peak_analysis_data, title=""):
    fig, ax =plt.subplots(1,2)
    fig.suptitle(title)
    ax[0].set_title('subject peaks')
    ax[1].set_title('click peaks')
    sns.boxplot(data=list(peak_analysis_data['subject_peaks'].values()),ax=ax[0])
    sns.boxplot(data=list(peak_analysis_data['click_peaks'].values()),ax=ax[1])

In [ ]:
plot_peak_analysis(peak_analysis_x, 'X Peaks')

In [ ]:
plot_peak_analysis(peak_analysis_y, 'Y Peaks')

In [ ]:
plot_peak_analysis(peak_analysis_z, 'Z Peaks')

In [ ]:
plot_peak_analysis(peak_analysis_mag, 'MAG Peaks')